In [6]:
!git clone https://github.com/KutuDev/DataCentricAIComp  # colne github repo

Cloning into 'DataCentricAIComp'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 66 (delta 31), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [7]:
def unzipper(in_folderPath, out_folderPath):  # folder unzip function
    import tarfile

    file = tarfile.open(in_folderPath)
    file.extractall(out_folderPath)
    file.close()

In [15]:
unzipper(in_folderPath='/content/DataCentricAIComp/data_v3.tar.gz', out_folderPath='./images')  #unzip files from github into the notebook

In [52]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

In [82]:
 pat=("/content/images/Users/Samuel/Desktop/data_v3/train")  #the path where the data set is stored on my google colab notebook (for the train set)
 os.mkdir("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/train/")#making a train folder on my google drive where images would be saved
 for filename in os.listdir(pat): #getting the folders in the file path
   folder=os.path.join(pat,filename)  #joining the folder names to the file path so we can go into the folders
   os.mkdir("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/train/"+ filename)  #making a directory to store the uninverted images in my google drive account,
   for files in os.listdir(folder): #getting the files in the folders
    file=os.path.join(folder,files)  #adding the file names to the file path so our uninvert function can access them directly uninvert directly
    im=Image.open(file) # our images are accessed directly
    im_invert = ImageOps.invert(im)  # images are uninverted
    im_invert.save("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/train/"+ filename + '/' + files) #uninverted images are stored in the folder i created in my google drive account

In [104]:
pat_val=("/content/images/Users/Samuel/Desktop/data_v3/val")  #repeated the procedures above for the val set changing only the file paths.
os.mkdir("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/val/")
for filename in os.listdir(pat_val):
   folder=os.path.join(pat_val,filename)
   os.mkdir("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/val/" + filename)
   for files in os.listdir(folder):
    file=os.path.join(folder,files)
    im=Image.open(file)
    im_invert = ImageOps.invert(im)
    im_invert.save("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/val/"+ filename + '/' + files)

In [55]:
'''for filename in os.listdir(path):
  k=os.path.join(path, filename)
  im=Image.open(k)
  im_invert = ImageOps.invert(im)
  im_invert.save("/content/images/Users/Samuel/Desktop/data_v3/train_1/i/"+ filename)'''

In [79]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import sys

directory = "/content/images"
user_data = directory + "/data_v4"
valid_data = directory + "/data_v4"
test_data = directory + "/label_book" # this can be the label book, or any other test set you create

### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
batch_size = 8
tf.random.set_seed(123)


if __name__ == "__main__":
    train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    valid = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()
    if total_length > 10_000:
        print(f"Dataset size larger than 10,000. Got {total_length} examples")
        sys.exit()

    test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    base_model = tf.keras.applications.ResNet50(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None,
    )
    base_model = tf.keras.Model(
        base_model.inputs, outputs=[base_model.get_layer("conv2_block3_out").output]
    )

    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(10)(x)
    model = tf.keras.Model(inputs, x)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    model.summary()
    loss_0, acc_0 = model.evaluate(valid)
    print(f"loss {loss_0}, acc {acc_0}")

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train,
        validation_data=valid,
        epochs=100,
        callbacks=[checkpoint],
    )

    model.load_weights("best_model")

    loss, acc = model.evaluate(valid)
    print(f"final loss {loss}, final acc {acc}")

    test_loss, test_acc = model.evaluate(test)
    print(f"test loss {test_loss}, test acc {test_acc}")

Found 1892 files belonging to 10 classes.
Found 1892 files belonging to 10 classes.
Found 52 files belonging to 10 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 32, 32, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 32, 32, 3)         0         
_________________________________________________________________
model (Functional)           (None, 8, 8, 256)         229760    
_________________________________________________________________
global_average_pooling2d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                2570      
Total params: 232,330
Trainable params: 229,386
Non-trainable params: 2,944
_________________________________________________